<a href="https://colab.research.google.com/github/PimiYvan/nlp_mnb_logreg/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [ ]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [ ]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [ ]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
def softmax(x):
    ### FILL CODE
    z = np.exp(x - x.max())/(np.sum(np.exp(x - x.max())))
    return z

In [ ]:
logits = np.random.rand(1, 20) + 100
# print(logits)
softmax(logits), np.sum(softmax(logits))

(array([[0.05385758, 0.0433586 , 0.05255862, 0.03334561, 0.06900721,
         0.0550396 , 0.02774372, 0.04837155, 0.05621772, 0.05306151,
         0.04265348, 0.03329527, 0.06479404, 0.07310627, 0.06139341,
         0.05932702, 0.03346754, 0.04956797, 0.03017234, 0.05966093]]),
 0.9999999999999998)

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [ ]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    lr = 0.1

    for iter in range(niter):
        for i in range(len(data)):
            x = data[i][1].reshape(-1, 1)
            y_label = data[i][0]
            y = np.zeros(nlabels)

            y[y_label] = 1
            y = y.reshape(-1, 1)

            # n_t = sigmoid(y@(w.dot(x).T))
            # tmp = (1 - n_t) @ y.dot(x.T)

            # n_t = sigmoid(y*(w.dot(x)))
            n_t = softmax(y*(w.dot(x)))
            tmp = ((1 - n_t) * y).dot(x.T)
            w = w + lr * tmp
        
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [ ]:
def predict(w, x):
    ## FILL CODE
    return w @ x

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [ ]:
def compute_accuracy(w, valid_data):
    result = 0
    for data in valid_data:
        pred = predict(w, data[1])
        res = np.argmax(softmax(pred))
        if res == data[0]:
            result += 1
    return result / len(valid_data)

In [ ]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)

w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print(w)



** Logistic Regression **

[[4.33186863e+00 2.96957166e-01 5.90121879e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.49458338e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.19998821e-01 1.13409656e-03]]


In [ ]:
w.shape

(10, 5826)

In [ ]:
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


Validation accuracy: 0.908

